In [13]:
# Module Importations
import numpy as np
import pandas as pd
import sklearn
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

# Print versioning information
print(sklearn.__version__)

0.22.2.post1


In [14]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.features import hyperparameter_opt
from src.models import model_evaluation
from src.models import sklearn_helpers

In [15]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [16]:
# Split data into training / evaluation sets
train_set, eval_set = split_data.split_train_test(original_dataset, 0.8)

In [17]:
# Drop profile id data column (both sets)
train_set = train_set.drop("profile_id", axis = 1)

eval_set = eval_set.drop("profile_id", axis = 1)

In [18]:
# Create torque target datasets
torque_train_data = train_set.drop("torque", axis = 1)
torque_label_data = train_set["torque"].copy()

torque_eval_data = eval_set.drop("torque", axis = 1)
torque_eval_label = eval_set["torque"].copy()

In [19]:
# Lin Reg Model (Target: Torque, Data: All)
lin_reg = LinearRegression()
lin_reg.fit(torque_train_data, torque_label_data)

# Name Model
model_type = "LinRegr_Torque"
model_name = sklearn_helpers.name_model(model_type)

# Save Model
sklearn_helpers.save_model(lin_reg, model_name)

# Compute RMSE via cross validation
scores_mse = cross_val_score(lin_reg, torque_train_data, torque_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg, torque_train_data, torque_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print cross validation results
model_name = "Linear Regression"
print(model_name, "rmse mean (cv):", scores_rmse.mean())
print(model_name, "mae mean (cv):", scores_mae.mean())

# Evaluate model
start_time = time.process_time()
torque_pred_eval = lin_reg.predict(torque_eval_data)
end_time = time.process_time()
print("Prediction Time:", (end_time - start_time))

model_evaluation.evaluate_model(model_name, torque_eval_label, torque_pred_eval)

Save Path: C:\Developer\electric_motor_thermal_modelling\Models\YC_LinRegr_Torque_2020_09_10-16_43_15.pkl
Linear Regression rmse mean (cv): 0.06137364641945211
Linear Regression mae mean (cv): 0.041007579675782946
Prediction Time: 0.015625
Linear Regression rmse (Eval): 0.0617094388532186
Linear Regression mae (Eval): 0.040985584127677104
Linear Regression r2 (Eval): 0.9961747621952894
